# Studing Chapter n.7 - Image Recognition Tasks Examples

## Reference's Book:
 - Deep Learning with PyTorch
   - by Eli Stevens, Luca Antiga, and Thomas Viehmann

## Check whether CUDA-like device is available and free to be used

In [1]:
import torch
device = (torch.device('cuda') if torch.cuda.is_available()
    else torch.device('gpu'))
print(f"Training on device {device}.")
print(f"# cuda device: {torch.cuda.device_count()}")
print(f"Id current device: {torch.cuda.current_device()}")

Training on device cuda.
# cuda device: 1
Id current device: 0


## Manually Download Data for any desired Reason

In [2]:
# !wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

In [3]:
 # !tar -xf /content/cifar-10-python.tar.gz

## Imports

In [4]:
import datetime
import torch

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets
from torchvision import transforms

In [5]:
dir(transforms)

['CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'functional',
 'functional_pil',
 'functional_tensor',
 'transforms']

## Custom Classes

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes = 2):
        super().__init__()
        
        # First Conv + Act + Pool full block
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        
        # Second Conv + Act + Pool full block
        self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


In [7]:
class NetWidth(nn.Module):
    def __init__(self, num_classes = 2, n_chans1 = 32):
        super().__init__()
        self.n_chans1 = n_chans1

        # First Conv Layer
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
        
        # Second Conv Layer
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size = 3, padding = 1)

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


In [8]:
class NetBatchNorm(nn.Module):
    def __init__(self, num_classes = 2, n_chans1 = 32):
        super().__init__()
        self.n_chans1 = n_chans1

        # First Conv Layer + BatchNorm full block
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features = n_chans1)
        
        # Second Conv Layer + BatchNorm full block
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size = 3, padding = 1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features = n_chans1 // 2)

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)

        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)

        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


In [9]:
class NetDepth(nn.Module):
    def __init__(self, num_classes = 2, n_chans1 = 32):
        super().__init__()
        self.n_chans1 = n_chans1


        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size = 3, padding = 1)
        

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out

        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


In [10]:
class NetDepth(nn.Module):
    def __init__(self, num_classes = 2, n_chans1 = 32):
        super().__init__()
        self.n_chans1 = n_chans1


        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size = 3, padding = 1)
        

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out

        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


In [11]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size = 3, padding = 1, bias = False)
        self.batch_norm = nn.BatchNorm2d(num_features = n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)
        pass

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x
    pass


class NetResDeep(nn.Module):
    def __init__(self, num_classes = 2, n_chans1 = 32, n_blocks = 10):
        super().__init__()
        self.n_chans1 = n_chans1


        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size = 3, padding = 1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans = n_chans1)])
        )

        # Fully Connected layers on top NN Arch.
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, num_classes)
        pass

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)

        out = F.max_pool2d(out, 2)
        
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    pass


## Functions

In [12]:
def training_loop(model, loss_fn, optimizer):
    """
    Basic training loop for brief tests
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data

    Returns:
    --------
    None 
    """
    
    for epoch in range(1, n_epochs + 1):
        for img, label in cifar2:
            out = model(img.view(-1).unsqueeze(0))
            loss = loss_fn(out, torch.tensor([label]))
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        pass


In [13]:
def convnet_validate(model, train_loader, val_loader):
    """
    Validate model's performance calculating Accuracy scores for both training
    set and validation set, where input model's is made of at least one convolution layer.

    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set

    Returns:
    --------
    :result_str: str Python object for later dispalying containing as a message 
        the Accuracy scores calculated for both train and val set passed independently to model instance 
    """

    # switch to evaluate mode
    model.eval()

    result_str = ""

    for name, loader in [('train', train_loader), ('val', val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
                pass
        result_str += "Accuracy {}: {:.5f} ".format(name, correct / total)
    
    return result_str


In [14]:
def batched_training_loop(model, loss_fn, optimizer, train_loader, val_loader):
    """
    Basic training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()

        for imgs, labels in train_loader:
            batch_size = imgs.shape[0]
            outputs = model(imgs.view(batch_size, -1))
            loss = loss_fn(outputs, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))


        # switch to evaluate mode
        model.eval()
        
        with torch.no_grad():
            for imgs, labels in loader:
                batch_size = imgs.shape[0]
                
                outputs = model(imgs.view(batch_size, -1))
                _, predicted = torch.max(outputs, dim = 1)
                
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
                pass
        print("Accuracy: %f" % (correct / total, ))
        pass


In [15]:
def basic_batched_convnet_training_loop(model, loss_fn, optimizer, train_loader, val_loader):
    """
    Basic training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth, 
    where model's arch is represented by a ConvNet.
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

        correct = 0
        total = 0

        # switch to evaluate mode
        model.eval()
        
        with torch.no_grad():
            for imgs, labels in loader:
                batch_size = imgs.shape[0]
                
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
                pass
        print("Accuracy: %f" % (correct / total, ))
        pass


In [16]:
def batched_convnet_training_loop(model, loss_fn, optimizer, train_loader, val_loader):
    """
    More Advanced training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth, 
    where model's arch is represented by a ConvNet.
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()
        
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Visualization data
            if step % 10 == 0:
                vis.plot_loss(np.mean(loss_values), step)
                loss_values.clear()
            pass

        result_validate_str = convnet_validate(model, train_loader, val_loader)
        
        epoch_msg = "%s Epoch: %d, Trainin Loss: %.5f %s" \
              % (str(datetime.datetime.now()),
                 epoch, float(loss),
                 result_validate_str.strip()
                 )
        print(epoch_msg)
        pass

def l2_reg_batched_convnet_training_loop(model, loss_fn, optimizer, train_loader, val_loader, l2_lambda = 1e-3):
    """
    More Advanced training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth, 
    where model's arch is represented by a ConvNet. In particular, here, it is introduced l2-Norm Regularization
    for controlling size of weight values which are all together model's parameters.
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set
    :l2_lambda: default to 1e-3, it represent the amount of l2-norm contribute to be accounted during weights update along backward pass

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_norm = sum(p.pow(2.0).sum()
                for p in model.parameters())
            loss = loss + l2_norm * l2_lambda
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass

        result_validate_str = convnet_validate(model, train_loader, val_loader)
        
        epoch_msg = "%s Epoch: %d, Trainin Loss: %.5f %s" \
              % (str(datetime.datetime.now()),
                 epoch, float(loss),
                 result_validate_str.strip()
                 )
        print(epoch_msg)
        pass

def l1_reg_batched_convnet_training_loop(model, loss_fn, optimizer, train_loader, val_loader, l1_lambda = 1e-3):
    """
    More Advanced training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth, 
    where model's arch is represented by a ConvNet. In particular, here, it is introduced l1-Norm Regularization
    for controlling size of weight values which are all together model's parameters.
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set
    :l1_lambda: default to 1e-3, it represent the amount of l1-norm contribute to be accounted during weights update along backward pass

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l1_norm = sum(p.abs().sum()
                for p in model.parameters())
            loss = loss + l1_norm * l1_lambda
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass

        result_validate_str = convnet_validate(model, train_loader, val_loader)
        
        epoch_msg = "%s Epoch: %d, Trainin Loss: %.5f %s" \
              % (str(datetime.datetime.now()),
                 epoch, float(loss),
                 result_validate_str.strip()
                 )
        print(epoch_msg)
        pass

def l1l2_reg_batched_convnet_training_loop(model, loss_fn, optimizer, train_loader, val_loader, l1_lambda = 1e-3, l2_lambda = 1e-3):
    """
    More Advanced training loop for brief tests exploiting Batch Strategy to let training be more stable and smooth, 
    where model's arch is represented by a ConvNet. In particular, here, it is introduced l1-Norm Regularization
    for controlling size of weight values which are all together model's parameters.
    
    Parameters:
    -----------
    :model: either nn.Sequential or subclass from nn.Module
    :loss_fn: loss function pytorch's instance, expressing a particular loss function shape to be minimized by means of a particular optimization strategy
    :optimizer: optimizer pytorch's instance representing the selected optimization strategy to be followed to fit the model's arch to the train data
    :train_loader: either nn.DataLoader or nn.DataSet instances collecting data examples employed for training the model
    :val_loader: either nn.DataLoader or nn.DataSet instances collecting data examples constituting validation set
    :l1_lambda: default to 1e-3, it represent the amount of l1-norm contribute to be accounted during weights update along backward pass
    :l2_lambda: default to 1e-3, it represent the amount of l2-norm contribute to be accounted during weights update along backward pass

    Returns:
    --------
    None 
    """
    for epoch in range(1, n_epochs + 1):

        # switch to train mode
        model.train()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l1_norm = sum(p.abs().sum()
                for p in model.parameters())
            l2_norm = sum(p.pow(2.0).sum()
                for p in model.parameters())
            loss = loss + l1_norm * l1_lambda + l2_norm * l2_lambda
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pass

        result_validate_str = convnet_validate(model, train_loader, val_loader)
        
        epoch_msg = "%s Epoch: %d, Trainin Loss: %.5f %s" \
              % (str(datetime.datetime.now()),
                 epoch, float(loss),
                 result_validate_str.strip()
                 )
        print(epoch_msg)
        pass


## Data

### Fetch Data

In [17]:
torch.manual_seed(0)
np.random.seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [18]:
data_path = '/content/cifar-10-batches-py'

In [19]:
class_objects = "airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck".split(",")

class_indeces = range(0, 10)
item_pairs = zip(class_indeces, class_objects)
class_names = dict(item_pairs)

class_indeces = range(0, 10)
item_pairs_reverse = zip(class_objects, class_indeces)
class_names_reverse = dict(item_pairs_reverse)

In [20]:
# Dowload CIFAR-10 Dataset
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

Extracting /content/cifar-10-batches-py/cifar-10-python.tar.gz to /content/cifar-10-batches-py
Files already downloaded and verified


In [21]:
type(cifar10).__mro__

(torchvision.datasets.cifar.CIFAR10,
 torchvision.datasets.vision.VisionDataset,
 torch.utils.data.dataset.Dataset,
 object)

In [22]:
type(cifar10_val).__mro__

(torchvision.datasets.cifar.CIFAR10,
 torchvision.datasets.vision.VisionDataset,
 torch.utils.data.dataset.Dataset,
 object)

In [23]:
print(f"CIFAR-10 dataset's size for training: {len(cifar10)}")
print(f"CIFAR-10 dataset's size for validation: {len(cifar10_val)}")

CIFAR-10 dataset's size for training: 50000
CIFAR-10 dataset's size for validation: 10000


### Builidng CIFAR-2 made out of just Bird and Airplane examples

In [24]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=True,
    transform=transforms.ToTensor())

imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)

mean_by_channels = imgs.view(3, -1).mean(dim=1)
std_by_channels = imgs.view(3, -1).std(dim=1)

Files already downloaded and verified


In [25]:
transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
    transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean_by_channels, std=std_by_channels)
            ]
        )
    )

transformed_cifar10_val = datasets.CIFAR10(data_path, train=False, download=True,
    transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean_by_channels, std=std_by_channels)
            ]
        )
    )

Files already downloaded and verified


In [26]:
label_map = {class_names_reverse['airplane']: 0, class_names_reverse['bird']: 1}
class_names = ['airplane', 'bird']

cifar2 = [
          (img, label_map[label])
          for img, label in transformed_cifar10
          if label in [0, 2]
]

cifar2_val = [
          (img, label_map[label])
          for img, label in transformed_cifar10_val
          if label in [0, 2]
]

## Perform Training Loop w/ nn.Sequential-like Instance

### LogSoftmax instead of Softmax - Test w/Modular Pytorch API

In [27]:
model = nn.Sequential(
    nn.Linear(3072, 512), nn.Tanh(),
    nn.Linear(512, 2), nn.LogSoftmax(dim=1)
)

In [28]:
model

Sequential(
  (0): Linear(in_features=3072, out_features=512, bias=True)
  (1): Tanh()
  (2): Linear(in_features=512, out_features=2, bias=True)
  (3): LogSoftmax(dim=1)
)

In [29]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

(1574402, [1572864, 512, 1024, 2])

In [30]:
loss = nn.NLLLoss()

In [31]:
img, label = cifar2[0]
out = model(img.view(-1).unsqueeze(0))

loss(out, torch.tensor([label]))

tensor(0.7730, grad_fn=<NllLossBackward>)

In [32]:
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [33]:
n_epochs = 100

In [34]:
training_loop(model, loss, optimizer)

Epoch: 1, Loss: 3.649296
Epoch: 2, Loss: 8.248081
Epoch: 3, Loss: 5.621278
Epoch: 4, Loss: 5.695742
Epoch: 5, Loss: 6.871593
Epoch: 6, Loss: 11.875392
Epoch: 7, Loss: 7.903219
Epoch: 8, Loss: 8.908370
Epoch: 9, Loss: 21.200356
Epoch: 10, Loss: 10.293543
Epoch: 11, Loss: 3.261814
Epoch: 12, Loss: 5.449851
Epoch: 13, Loss: 9.478970
Epoch: 14, Loss: 11.130045
Epoch: 15, Loss: 1.790611
Epoch: 16, Loss: 1.838362
Epoch: 17, Loss: 8.642748
Epoch: 18, Loss: 7.640148
Epoch: 19, Loss: 4.509187
Epoch: 20, Loss: 5.853117
Epoch: 21, Loss: 3.637940
Epoch: 22, Loss: 11.633747
Epoch: 23, Loss: 6.734209
Epoch: 24, Loss: 11.464343
Epoch: 25, Loss: 1.820391
Epoch: 26, Loss: 13.453730
Epoch: 27, Loss: 0.086635
Epoch: 28, Loss: 4.250389
Epoch: 29, Loss: 0.020550
Epoch: 30, Loss: 1.623963
Epoch: 31, Loss: 3.981997
Epoch: 32, Loss: 3.133183
Epoch: 33, Loss: 0.563246
Epoch: 34, Loss: 9.677677
Epoch: 35, Loss: 13.629996
Epoch: 36, Loss: 14.910199
Epoch: 37, Loss: 7.402074
Epoch: 38, Loss: 10.307061
Epoch: 39, 

### Minibatch Strategy for Updating in a wiser way Model's Parameters

In [35]:
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.NLLLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

### Somewhat Arbitrarily Deeper Model - LogSoftmax + NLLLoss - Test w/ Modular Pytorch API

In [36]:
model = nn.Sequential(
    nn.Linear(3072, 1024), nn.Tanh(),
    nn.Linear(1024, 512), nn.Tanh(),
    nn.Linear(512, 128), nn.Tanh(),
    nn.Linear(128, 2), nn.LogSoftmax(dim=1)
)

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.NLLLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [37]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]

In [38]:
sum(numel_list), numel_list

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])

In [39]:
batched_training_loop(model, loss, optimizer, train_loader, val_loader)

Epoch: 1, Loss: 0.448006


NameError: ignored

### Somewhat Arbitrarily Deeper Model - CrossEntropyLoss - Test w/ Modular Pytorch API

In [ ]:
model = nn.Sequential(
    nn.Linear(3072, 1024), nn.Tanh(),
    nn.Linear(1024, 512), nn.Tanh(),
    nn.Linear(512, 128), nn.Tanh(),
    nn.Linear(128, 2)
)

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]

In [ ]:
sum(numel_list), numel_list

In [ ]:
batched_training_loop(model, loss, optimizer, train_loader, val_loader)

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'fc_birds_vs_airplane.pt')

### Convolutional Neural Network Example - Modular Pytorch API

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size = 3, padding = 1), nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 8, kernel_size = 3, padding = 1), nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(8 * 8 * 8, 32), nn.Tanh(),
    nn.Linear(32, 2)
)

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]

In [ ]:
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'convnet_birds_vs_airplane.pt')

In [ ]:
"""loaded_model = model.load_state_dict(
    torch.load(
        data_path + 'convnet_birds_vs_airplane.pt',
        map_location = device
    )
)
"""

## Perform Training Loop w/ subclassing nn.Module

### Training by means of an instance of subclassed nn.Module Net class

In [ ]:
model = Net()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'Net_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'Net_birds_vs_airplane.pt')

### Setting Width of first Conv Layer via an instance of NetWidth Class w/ Functional API exploited for Pool Layer

In [ ]:
model = NetWidth()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'NetWidth_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'NetWidth_birds_vs_airplane.pt')

### BatchNorm Network Model

In [ ]:
model = NetBatchNorm()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'NetBatchNorm_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'NetBatchNorm_birds_vs_airplane.pt')

In [ ]:
model = NetDepth()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'NetDepth_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'NetDepth_birds_vs_airplane.pt')

## Perform Training Loop w/ subclassing nn.Module + L2 or L1 or Both Regularizations

In [ ]:
model = NetBatchNorm()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
l2_reg_batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'NetBatchNorm_l2_norm_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'NetBatchNorm_l2_norm_birds_vs_airplane.pt')

In [ ]:
model = NetBatchNorm()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
l1_reg_batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
model = NetBatchNorm()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
l1l2_reg_batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

## ResNet-like NN model - *Test*

In [ ]:
model = NetResDeep()

n_epochs = 100
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

loss = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    cifar2, batch_size = 64, shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    cifar2_val, batch_size = 64, shuffle = False
)

In [ ]:
model

In [ ]:
numel_list = [p.numel() for p in model.parameters() if p.requires_grad == True]
sum(numel_list), numel_list

In [ ]:
batched_convnet_training_loop(model.to(device = device), loss, optimizer, train_loader, val_loader)

In [ ]:
data_path = '/content/'
torch.save(model.state_dict(), data_path + 'ResNet_norm_batchnorm_birds_vs_airplane.pt')

In [ ]:
# data_path = '/content/'
# torch.save(model.state_dict(), data_path + 'ResNet_norm_batchnorm_birds_vs_airplane.pt')

## References

- Pytorch Reference's Manual:
 - [torch.nn module](https://pytorch.org/docs/stable/nn.html)

- Initialization Topic (Papers):
  - [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) by X. Glorot & Y.Bengio, which lead to default Pytorch's weights initialization knwon as *Xavier initialization* algorithm or scheme
 -  [Fixup Initialization: Residual Learning Without Normalization](https://arxiv.org/abs/1901.09321) by Hongyi Zhang, Yann N. Dauphin, Tengyu Ma, whose works allows to *get rid off batch normalization layers* with a given particular NN Arch, to still be able to train a NN arch with meaningful and confident results or performance.

- Activation Functions (Papers):
  - [Deep Learning using Rectified Linear Units (ReLU)](https://arxiv.org/pdf/1803.08375.pdf)

- Datasets:
  - [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)

- Regularization techniques (Papers):
  - [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shif](https://arxiv.org/abs/1502.03167)
  - [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)

- Archs Types (Papers):
  - [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
  - [Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)
  - [Highway Networks
](https://arxiv.org/pdf/1505.00387.pdf)
  - [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)

- Some Third Party useful Tutorials:
 - [Imagenet example](https://github.com/pytorch/examples/blob/master/imagenet/main.py#L327)
 - [Writing a better code with pytorch and einops](https://arogozhnikov.github.io/einops/pytorch-examples.html)

- Books
  - [List of books for improving Pytorch knowledge](https://bookauthority.org/books/best-pytorch-books)